# Разработка прототипа рекомендательной системы

## Импорт библиотек

In [75]:
import pandas as pd
import numpy as np
import re
import ast
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import plotly.express as px

from sentence_transformers import SentenceTransformer
import torch
from sklearn.metrics.pairwise import cosine_similarity

from docx import Document

In [79]:
import plotly.express as px

In [2]:
MODEL_ID = "ai-forever/sbert_large_nlu_ru"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = SentenceTransformer(MODEL_ID, device=device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

## Загрузка данных

In [3]:
DATA_DIR = Path("../data")
PROCESSED_DIR = DATA_DIR / "processed"
RESUME_DIR = DATA_DIR / "resume"

EXTRACTED_CSV = PROCESSED_DIR / "extracted_skills.csv"
EMB_PARQUET = PROCESSED_DIR / "embeddings.parquet"
RESUME_PATH = RESUME_DIR / "resume_junior_ds.docx"

print(EXTRACTED_CSV)
print(EMB_PARQUET)
print(RESUME_PATH)

..\data\processed\extracted_skills.csv
..\data\processed\embeddings.parquet
..\data\resume\resume_junior_ds.docx


In [28]:
# 1. Загружаем базу вакансий
skills_df = pd.read_csv(EXTRACTED_CSV, dtype={"id": str})
emb_df = pd.read_parquet(EMB_PARQUET)

print(skills_df.shape, emb_df.shape)


(677, 3) (677, 3)


In [29]:
desc_emb = np.vstack(emb_df["desc_emb"].apply(np.array).values)
skills_emb = np.vstack(emb_df["skills_emb"].apply(np.array).values)

desc_emb.shape, skills_emb.shape

((677, 1024), (677, 1024))

In [30]:
# Создаем список уникальных навыков

skills_df["skills_list"] = skills_df["skills_list"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])


vocab = (
    skills_df["skills_list"]
    .explode()            # раскрыть списки
    .dropna()
    .unique()
    .tolist()
)
vocab = sorted(vocab)

len(vocab), vocab[:30]


(279,
 ['a/b тесты',
  'a/b/n',
  'ai',
  'airflow',
  'amplitude',
  'analysis',
  'anomaly detection',
  'apache airflow',
  'apache kafka',
  'apache spark',
  'apache superset',
  'api',
  'appsflyer',
  'argo workflows',
  'audio ml',
  'aws',
  'bash',
  'bert',
  'bi',
  'big data',
  'bigquery',
  'bpmn',
  'business intelligence',
  'cac',
  'catboost',
  'chatgpt',
  'ci/cd',
  'ci\\cd',
  'classicml libs',
  'clickhouse'])

In [15]:
# 2. Читаем резюме из docx

def read_docx(path: Path) -> str:
    doc = Document(path)
    texts = [p.text for p in doc.paragraphs if p.text.strip()]
    return "\n".join(texts)

resume_text = read_docx(RESUME_PATH)
resume_text


'Игорь Иванов — Junior Data Scientist\nКонтакты\nEmail: example@email.com\nТелефон: +7 (900) 000-00-00\nГород: Москва\nО себе\nНачинающий специалист по анализу данных. Обладаю базовыми навыками в машинном обучении, работе с данными и построении аналитических моделей. Интересуюсь разработкой рекомендательных систем и применением NLP в бизнес-задачах.\nНавыки\n• Python (pandas, numpy, matplotlib, scikit-learn)\n• SQL (PostgreSQL, MySQL)\n• Machine Learning (классические модели)\n• Работа с API\n• Data Visualization (Power BI, Tableau)\n• Git, Linux, Jupyter Notebook\nОпыт проектов\nАнализ продаж — исследование данных, визуализация ключевых метрик, построение прогностической модели.\nКлассификация текстов — подготовка выборки, предобработка, обучение модели Logistic Regression.\nОбразование\nБакалавр, Прикладная математика и информатика\nУниверситет — 2020–2024'

 ## Извлечение навыков из текста резюме, векторизация резюме и навыков, сравнение с описанием вакансии

In [19]:
def extract_resume_skills(text: str, vocab) -> list[str]:
    """
    Находит навыки из vocab, которые упоминаются в тексте резюме.
    """
    text_norm = text.lower()
    found = set()

    for skill in vocab:
        s = str(skill).strip().lower()
        if not s:
            continue

        # фильтруем шум
        if len(s) == 1 and s != "r":
            continue
        if s == "it":
            continue

        # одно слово — ищем по границам слова; фразы — как подстроку
        if " " in s:
            pattern = re.escape(s)
        else:
            pattern = r"\b" + re.escape(s) + r"\b"

        if re.search(pattern, text_norm):
            found.add(s)

    return sorted(found)


resume_skills = extract_resume_skills(resume_text, vocab)
len(resume_skills), resume_skills[:20]

(18,
 ['api',
  'bi',
  'data scientist',
  'git',
  'jupyter notebook',
  'linux',
  'machine learning',
  'matplotlib',
  'mysql',
  'nlp',
  'numpy',
  'pandas',
  'postgresql',
  'power bi',
  'python',
  'scikit-learn',
  'sql',
  'tableau'])

In [20]:
# Векторизуем резюме и навыки, извлеченные из резюме

model = SentenceTransformer(MODEL_ID, device=device)

# 1) Эмбеддинг полного текста резюме
resume_emb = model.encode(
    [resume_text],
    normalize_embeddings=True
)

# 2) Эмбеддинги навыков из резюме (строковый список)
resume_skill_embs = model.encode(
    resume_skills,
    normalize_embeddings=True
)

resume_emb.shape, resume_skill_embs.shape


((1, 1024), (18, 1024))

In [21]:
# По косинусному расстоянию находим топ-20 наиболее подходящих вакансий

sim_desc = cosine_similarity(resume_emb, desc_emb)[0]

# ТОП-20 похожих вакансий
top_idx = np.argsort(sim_desc)[::-1][:20]


similar_vacancies = skills_df.iloc[top_idx][["id", "description", "skills_list"]]
similar_vacancies


,id,description,skills_list
509,127543165,🏢 О компании PureCommit (Пьюркоммит) — компани...,"[ai, api, ci/cd, data science, docker, e-comme..."
125,126381134,"Обязанности: Проведение исследование данных, э...","[catboost, ci/cd, fastapi, flask, kafka, light..."
79,127041612,BELL INTEGRATOR – крупный системный интегратор...,"[deep learning, ml, mlops, nlp, pyspark, pytho..."
401,127401542,О проекте:«Нейроллаб» — исследовательская нейр...,"[ci/cd, dl, docker, matplotlib, ml, mne-python..."
175,126700452,"Стек: Oracle, ClickHouse, Docker, Python, Fast...","[airflow, clickhouse, dl, docker, fastapi, git..."
621,127855846,"Стек: Oracle, ClickHouse, Docker, Python, Fast...","[airflow, clickhouse, dl, docker, fastapi, git..."
86,124811999,Группа компаний «Спектрум» — динамичная инжини...,"[bi, clickhouse, datalens, google sheets, ms e..."
192,126924082,В команду по разработке перспективных алгоритм...,"[ds, git, lightgbm, matplotlib, numpy, ocr, pa..."
97,126533713,Команда ценообразования для Кредитов ЮЛ. Проек...,"[catboost, data scientist, hadoop, lightgbm, m..."
601,127826787,Консалтика — российская аккредитованная IT-ком...,"[airflow, api, bi, data engineer, dbt, dwh, et..."


In [32]:
# Векторизуем навыки из вакансий

vac_skill_emb = model.encode(vocab, normalize_embeddings=True)
vac_skill_emb.shape

(279, 1024)

In [35]:
# Строим матрицу сходства
sim_matrix = cosine_similarity(resume_skill_embs, vac_skill_emb)
sim_matrix.shape   # (resume_K, vacancy_M)


(18, 279)

In [37]:
# Определяем похожие навыки (все, что ближе 0.65)

threshold = 0.65

matched_vac_skills = set()
for i, skill in enumerate(resume_skills):
    sims = sim_matrix[i]
    close_idx = np.where(sims >= threshold)[0]
    for j in close_idx:
        matched_vac_skills.add(all_vac_skills[j])


In [38]:
# Определяем недостающие навыки, относительно "карты рынка"

missing_skills = sorted(set(all_vac_skills) - matched_vac_skills)
missing_skills[:20]


['a/b тесты',
 'rfm-анализ',
 'а/б тесты',
 'аб тестирование',
 'автоматизация процессов',
 'адекватность',
 'айтрекинг',
 'алгоритмы',
 'алгоритмы и структуры данных',
 'анализ бизнес-процессов',
 'анализ временных рядов',
 'анализ данных',
 'анализ рынка',
 'анализ требований',
 'анализ финансовых показателей',
 'аналитика',
 'аналитический склад ума',
 'аналитическое мышление',
 'базы данных',
 'бизнес-анализ']

In [39]:
# Определяем недостающие навыки, относительно "карты рынка", с учетом частоты упоминания в вакансиях

freq = (
    skills_df.explode("skills_list")["skills_list"]
    .value_counts()
    .to_dict()
)

missing_skills_sorted = sorted(missing_skills, key=lambda s: freq.get(s, 0), reverse=True)
missing_skills_sorted[:20]


['аналитика',
 'поиск',
 'ии',
 'алгоритмы',
 'анализ данных',
 'дашборды',
 'прогнозирование',
 'ответственность',
 'обучение и развитие',
 'аналитический склад ума',
 'ооп',
 'базы данных',
 'планирование',
 'проактивность',
 'статистика',
 'нейронные сети',
 'временные ряды',
 'аналитическое мышление',
 'контроль качества данных',
 'рекомендательные системы']

In [53]:
#Формируем таблицу рекомендаций по навыкам

# 1) Глобальные частоты навыков по всему датасету
global_freq = (
    skills_df
    .explode("skills_list")["skills_list"]
    .value_counts()
)

# 2) Локальные частоты по похожим вакансиям
local_freq = (
    similar_vacancies
    .explode("skills_list")["skills_list"]
    .value_counts()
)

# 3) Быстрая мапа skill -> индекс в all_vac_skills (столбцы sim_matrix)
skill_to_idx = {s: i for i, s in enumerate(all_vac_skills)}

rows = []
for skill in missing_skills:
    j = skill_to_idx.get(skill)
    if j is None:
        continue

    # максимальная похожесть этого навыка с любым навыком из резюме
    max_sim = float(sim_matrix[:, j].max()) if sim_matrix.size > 0 else 0.0

    rows.append({
        "skill": skill,
        "local_freq": int(local_freq.get(skill, 0)),
        "global_freq": int(global_freq.get(skill, 0)),
        "max_sim_to_resume": max_sim,
    })

reco_df = (
    pd.DataFrame(rows)
    .sort_values(
        ["local_freq", "global_freq", "max_sim_to_resume"],
        ascending=[False, False, False]
    )
    .reset_index(drop=True)
)

reco_df.head(20)


,skill,local_freq,global_freq,max_sim_to_resume
0,анализ данных,8,56,0.629090
1,аналитика,5,137,0.625329
2,поиск,5,100,0.506752
3,аналитический склад ума,4,26,0.505960
4,алгоритмы,3,62,0.643721
5,прогнозирование,3,39,0.598229
6,ответственность,3,32,0.457769
7,оптимизация процессов,3,8,0.632622
8,статистика,2,20,0.533447
9,разработка по,2,11,0.567581


In [56]:
# 1. Берём вектор резюме
resume_vec = resume_emb[0]  # (768,)

# 2. Готовим навыки ближайших вакансий
skills_flat = []
for lst in similar_vacancies["skills_list"]:
    skills_flat.extend(lst)

skills_flat = list(sorted(set(skills_flat)))

print(f"Уникальных навыков в похожих вакансиях: {len(skills_flat)}")

# 3. Векторизуем каждый навык как текст
skill_vectors = []
for s in skills_flat:
    v = model.encode(s, convert_to_numpy=True, normalize_embeddings=True)
    skill_vectors.append(v)

skill_vectors = np.array(skill_vectors)

# 4. Считаем близость навыков к резюме
skill_sim = cosine_similarity([resume_vec], skill_vectors)[0]

skill_df = pd.DataFrame({
    "skill": skills_flat,
    "similarity": skill_sim
})

skill_df.sort_values("similarity", ascending=False).head(10)

Уникальных навыков в похожих вакансиях: 107


,skill,similarity
81,алгоритмы машинного обучения,0.601875
39,machine learning,0.589232
79,автоматизация управленческой отчетности,0.587206
26,google data studio,0.575481
83,анализ финансовых показателей,0.573027
65,scikit-learn,0.565661
91,коммуникативные навыки,0.557541
7,business intelligence,0.554667
18,deep learning,0.551104
77,автоматизация отчётности,0.533207


In [68]:
def recommend_skills(
    resume_text: str,
    skills_df: pd.DataFrame,
    emb_df: pd.DataFrame,
    model,
    top_vac=20,
    top_skills=20,
):
    """
    Универсальная функция рекомендаций навыков по резюме.

    Параметры:
        resume_text: текст резюме
        skills_df: таблица extracted_skills.csv (id, description, skills_list)
        emb_df: таблица embeddings (id, desc_emb, skills_emb)
        model: SentenceTransformer
        top_vac: число похожих вакансий
        top_skills: число рекомендуемых навыков

    Возвращает:
        dict с результатами + красивый текст
    """

    # ---------- 1. Векторизация резюме ----------
    resume_emb = model.encode([resume_text], convert_to_numpy=True, normalize_embeddings=True)

    # ---------- 2. Похожесть с вакансиями по описанию ----------
    desc_emb = np.vstack(emb_df["desc_emb"].values)
    sim_desc = cosine_similarity(resume_emb, desc_emb)[0]

    top_idx = np.argsort(sim_desc)[::-1][:top_vac]
    similar_vac = skills_df.iloc[top_idx].copy()
    similar_vac["similarity"] = sim_desc[top_idx]

    # ---------- 3. Извлекаем все навыки из похожих вакансий ----------
    all_vac_skills = []
    for lst in similar_vac["skills_list"]:
        all_vac_skills.extend(lst)

    vc = pd.Series(all_vac_skills).value_counts()

    # ---------- 4. Извлекаем навыки резюме ----------
    resume_skills = extract_resume_skills(resume_text, vc.index.tolist())

    # ---------- 5. Находим недостающие ----------
    missing = list(set(vc.index) - set([s.lower() for s in resume_skills]))

    # ---------- 6. Считаем близость недостающих навыков к резюме ----------
    skill_texts = missing
    skill_emb = model.encode(skill_texts, convert_to_numpy=True, normalize_embeddings=True)

    sim_to_resume = cosine_similarity(resume_emb, skill_emb)[0]

    rec_df = pd.DataFrame({
        "skill": missing,
        "frequency": vc[missing].values,
        "sim_to_resume": sim_to_resume
    })

    # ранжируем: по частоте и similarity
    rec_df["score"] = (
        0.6 * (rec_df["frequency"] / rec_df["frequency"].max()) +
        0.4 * rec_df["sim_to_resume"]
    )

    recommendations = rec_df.sort_values("score", ascending=False).head(top_skills)

    # ---------- 7. Готовим красивый вывод ----------
    text = "### Рекомендации для улучшения резюме\n"
    text += f"Проанализировано похожих вакансий: **{top_vac}**\n\n"
    text += f"Навыков найдено в похожих вакансиях: **{len(vc)}**\n\n"
    text += f"Навыков в резюме: **{len(resume_skills)}**\n\n"
    text += "### Недостающие навыки (ранжированные)\n\n"

    for i, row in recommendations.iterrows():
        text += f"- **{row.skill}** — freq: {row.frequency}, similarity: {row.sim_to_resume:.3f}\n"

    text += "\n---\n"
    text += "⚡ Навыки выбраны исходя из:\n"
    text += "- частоты встречаемости в вакансиях\n"
    text += "- семантической близости к вашему резюме (через SBERT эмбеддинги)\n"

    return {
        "similar_vacancies": similar_vac,
        "resume_skills": resume_skills,
        "missing_skills": missing,
        "recommendations": recommendations,
        "text": text
    }



In [69]:
result = recommend_skills(
    resume_text,
    skills_df,
    emb_df,
    model,
    top_vac=20,
    top_skills=15
)

print(result["text"])



### Рекомендации для улучшения резюме
Проанализировано похожих вакансий: **20**

Навыков найдено в похожих вакансиях: **120**

Навыков в резюме: **17**

### Недостающие навыки (ранжированные)

- **ml** — freq: 13, similarity: 0.184
- **docker** — freq: 9, similarity: 0.373
- **pytorch** — freq: 8, similarity: 0.334
- **llm** — freq: 8, similarity: 0.224
- **xgboost** — freq: 6, similarity: 0.374
- **data science** — freq: 5, similarity: 0.482
- **lightgbm** — freq: 6, similarity: 0.262
- **ci/cd** — freq: 5, similarity: 0.343
- **airflow** — freq: 5, similarity: 0.339
- **tensorflow** — freq: 4, similarity: 0.352
- **dl** — freq: 5, similarity: 0.222
- **transformers** — freq: 3, similarity: 0.448
- **sklearn** — freq: 4, similarity: 0.327
- **hugging face** — freq: 3, similarity: 0.431
- **mlflow** — freq: 4, similarity: 0.313

---
⚡ Навыки выбраны исходя из:
- частоты встречаемости в вакансиях
- семантической близости к вашему резюме (через SBERT эмбеддинги)



In [71]:
# Попробуем решить проблему синонимов

# Явные синонимичные группы
EQUIV_GROUPS = [
    {"ml", "machine learning", "машинное обучение"},
    {"dl", "deep learning", "глубинное обучение"},
    {"ds", "data science", "data scientist", "дата саентист"},
]

def expand_synonyms(skills: set[str]) -> set[str]:
    """
    Если в резюме есть хотя бы один навык из группы,
    считаем, что все навыки группы тоже «покрыты».
    """
    result = {s.lower() for s in skills}
    for group in EQUIV_GROUPS:
        group_lower = {g.lower() for g in group}
        if result & group_lower:
            result |= group_lower
    return result

In [72]:
def recommend_skills(
    resume_text: str,
    model,
    skills_df: pd.DataFrame,
    emb_df: pd.DataFrame,
    top_n_vacancies: int = 20,
    top_n_skills: int = 20,
    skill_sim_threshold: float = 0.6,
):
    """
    Рекомендации по навыкам на основе:
    - схожести резюме с вакансиями (по desc_emb),
    - частоты навыков в похожих вакансиях,
    - семантической близости навыков к резюме (чтобы не рекомендовать то, что уже по смыслу есть).

    Возвращает словарь:
    {
      "resume_skills": set(...),          # навыки, найденные в резюме (с учетом синонимов)
      "similar_vacancies": DataFrame,     # топ-похожие вакансии
      "recommendations": DataFrame        # финальный список навыков к развитию
    }
    """
    # 1) вытаскиваем skills_list как список строк
    def parse_skills_cell(x):
        if isinstance(x, list):
            return x
        if isinstance(x, str):
            x = x.strip()
            if x.startswith("[") and x.endswith("]"):
                import ast
                try:
                    return [s.strip() for s in ast.literal_eval(x)]
                except Exception:
                    pass
            # запасной вариант: "a, b, c"
            return [s.strip() for s in x.split(",") if s.strip()]
        return []

    skills_lists = skills_df["skills_list"].apply(parse_skills_cell)

    # 2) словарь всех навыков из базы (vocab)
    vocab = sorted({s.lower() for lst in skills_lists for s in lst})

    # 3) извлекаем навыки из резюме по словарю (простой substring-match)
    resume_lower = resume_text.lower()
    resume_skills = sorted({v for v in vocab if v and v in resume_lower})

    # 4) расширяем резюме-скиллы синонимами (ml <-> machine learning и т.п.)
    resume_skills_full = expand_synonyms(set(resume_skills))

    # 5) эмбеддинг резюме
    resume_emb = model.encode([resume_text], normalize_embeddings=True)

    # 6) похожие вакансии по desc_emb
    desc_emb = np.vstack(emb_df["desc_emb"])
    sim_desc = cosine_similarity(resume_emb, desc_emb)[0]
    top_idx = np.argsort(sim_desc)[::-1][:top_n_vacancies]

    similar_vacancies = skills_df.iloc[top_idx].copy()
    similar_vacancies["similarity"] = sim_desc[top_idx]

    # 7) считаем частоты навыков в похожих вакансиях, игнорируя то,
    #    что уже есть в резюме (с учетом синонимов)
    counter = Counter()
    for lst in skills_lists.iloc[top_idx]:
        for s in lst:
            s_norm = s.lower()
            if s_norm in resume_skills_full:
                continue
            counter[s_norm] += 1

    if not counter:
        rec_df = pd.DataFrame(columns=["skill", "freq", "similarity"])
        return {
            "resume_skills": sorted(resume_skills_full),
            "similar_vacancies": similar_vacancies,
            "recommendations": rec_df,
        }

    # 8) список кандидатов-навыков и их эмбеддинги
    candidate_skills = list(counter.keys())
    skills_emb = model.encode(candidate_skills, normalize_embeddings=True)
    sim_skill = cosine_similarity(resume_emb, skills_emb)[0]

    # 9) фильтруем навыки, которые по смыслу и так уже близки к резюме
    rows = []
    for skill, sim_val in zip(candidate_skills, sim_skill):
        # если навык близок к резюме по эмбеддингу — считаем покрытым, не рекомендуем
        if sim_val >= skill_sim_threshold:
            continue
        rows.append({
            "skill": skill,
            "freq": counter[skill],
            "similarity": float(sim_val),
        })

    rec_df = pd.DataFrame(rows)
    if not rec_df.empty:
        rec_df = rec_df.sort_values(
            by=["freq", "similarity"], ascending=[False, False]
        ).head(top_n_skills)

    return {
        "resume_skills": sorted(resume_skills_full),
        "similar_vacancies": similar_vacancies,
        "recommendations": rec_df,
    }


In [73]:
result = recommend_skills(
    resume_text=resume_text,
    model=model,
    skills_df=skills_df,
    emb_df=emb_df,
    top_n_vacancies=20,
    top_n_skills=20,
    skill_sim_threshold=0.6,
)

resume_sk = result["resume_skills"]
rec_df = result["recommendations"]


In [74]:
print("Навыки, найденные в резюме:")
print(", ".join(sorted(resume_sk)))
print("\nРекомендации к развитию:")

if rec_df.empty:
    print("Все ключевые навыки по текущему профилю уже покрыты.")
else:
    for _, row in rec_df.iterrows():
        print(f"- {row['skill']} — частота в похожих вакансиях: {row['freq']}, "
              f"семантическая близость к резюме: {row['similarity']:.3f}")


Навыки, найденные в резюме:
ai, api, bi, data science, data scientist, ds, git, jupyter notebook, linux, machine learning, matplotlib, ml, mysql, nlp, numpy, pandas, postgres, postgresql, power bi, python, r, scikit-learn, sql, tableau, дата саентист, ии, машинное обучение

Рекомендации к развитию:
- docker — частота в похожих вакансиях: 9, семантическая близость к резюме: 0.373
- pytorch — частота в похожих вакансиях: 8, семантическая близость к резюме: 0.334
- llm — частота в похожих вакансиях: 8, семантическая близость к резюме: 0.224
- xgboost — частота в похожих вакансиях: 6, семантическая близость к резюме: 0.374
- lightgbm — частота в похожих вакансиях: 6, семантическая близость к резюме: 0.262
- ci/cd — частота в похожих вакансиях: 5, семантическая близость к резюме: 0.343
- airflow — частота в похожих вакансиях: 5, семантическая близость к резюме: 0.339
- dl — частота в похожих вакансиях: 5, семантическая близость к резюме: 0.222
- tensorflow — частота в похожих вакансиях: 4, 

In [81]:
def plot_skill_recommendations_plotly(rec_df, top_n=20):
    """
    Красивый интерактивный график рекомендаций:
    - по оси Y — навыки
    - по оси X — семантическая близость к резюме
    - размер пузырька — частота навыка в похожих вакансиях
    - цвет — тоже похожесть (similarity)
    """
    if rec_df.empty:
        print("Нет рекомендаций для отображения.")
        return

    df = (
        rec_df
        .copy()
        .sort_values(["freq", "similarity"], ascending=[False, False])
        .head(top_n)
    )

    fig = px.scatter(
        df,
        x="similarity",
        y="skill",
        size="freq",
        color="similarity",
        size_max=30,
        color_continuous_scale="Blues",
        labels={
            "similarity": "Семантическая близость к резюме",
            "freq": "Частота в похожих вакансиях",
            "skill": "Навык",
        },
        title="Рекомендованные навыки для развития",
        hover_data={
            "freq": True,
            "similarity": ":.3f",
            "skill": False,  # skill и так по оси Y
        },
    )

    # Чтобы самые важные навыки были сверху
    fig.update_yaxes(categoryorder="array", categoryarray=df.sort_values(
        ["freq", "similarity"], ascending=[True, True]
    )["skill"].tolist())

    fig.update_layout(
        xaxis=dict(range=[df["similarity"].min() - 0.02,
                          df["similarity"].max() + 0.02]),
        template="plotly_white",
        height=600,
        margin=dict(l=150, r=40, t=60, b=60),
    )

    fig.show()

# Вызов функции
plot_skill_recommendations_plotly(rec_df, top_n=20)
